In [1]:
import pandas as pd
# Read Excel file and insert data into the table
districts_year_df = pd.read_excel('data/SchoolDistricts.xlsx', sheet_name='Data')
districts_loc_df = pd.read_excel('data/SchoolDistrictsLocation.xlsx')

clean_districts_year_df = districts_year_df[['leaid', 'ppcstot', 'predcost', 'fundinggap', 'outcomegap', 'enroll', 'year', 'state_name']]
clean_districts_loc_df = districts_loc_df[['LEAID', 'NAME', 'CITY', 'LAT', 'LON']]

C:\Users\Beverly\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
print(clean_districts_year_df.head())
print(clean_districts_loc_df.head())

    leaid  ppcstot  predcost  fundinggap  outcomegap  enroll  year state_name
0  100190    10428  10815.29    -387.292         NaN    6293  2020    Alabama
1  100005     8663  11494.05   -2831.046         NaN    5824  2020    Alabama
2  100030     9685  17402.60   -7717.598         NaN    3036  2020    Alabama
3  100060     9214  15706.25   -6492.253         NaN    1839  2020    Alabama
4  100090    11996  22397.93  -10401.930         NaN    1887  2020    Alabama
    LEAID                    NAME          CITY        LAT        LON
0  100002  Alabama Youth Services      Mt Meigs  32.376397 -86.084642
1  100005        Albertville City   Albertville  34.282186 -86.223627
2  100006         Marshall County  Guntersville  34.305770 -86.286958
3  100007             Hoover City        Hoover  33.407169 -86.766082
4  100008            Madison City       Madison  34.686721 -86.745741


In [23]:
import sqlite3
import csv

conn = sqlite3.connect('example.db')
cursor = conn.cursor()

cursor.execute('''
    DROP TABLE IF EXISTS DistrictsYear;
    ''')

cursor.execute('''
    DROP TABLE IF EXISTS DistrictsLocation;
    ''')

cursor.execute('''CREATE TABLE DistrictsYear (
                    leaid INTEGER,
                    ppcstot INTEGER,
                    predcost INTEGER,
                    fundinggap INTEGER,
                    outcomegap INTEGER,
                    enroll INTEGER,
                    year INTEGER,
                    state_name TEXT
                 )''')

cursor.execute('''CREATE TABLE DistrictsLocation (
                    LEAID INTEGER,
                    NAME INTEGER,
                    CITY TEXT,
                    LAT INTEGER,
                    LON INTEGER
                 )''')

# insert into DistrictYear table
for row in clean_districts_year_df.itertuples(index=False):
    cursor.execute('''
        INSERT INTO DistrictsYear
        (leaid, ppcstot, predcost, fundinggap, outcomegap, enroll, year, state_name)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', 
        row)

# insert into DistrictLocation table
for row in clean_districts_loc_df.itertuples(index=False):
    cursor.execute('''
        INSERT INTO DistrictsLocation
        (LEAID, NAME, CITY, LAT, LON)
        VALUES (?, ?, ?, ?, ?)
        ''', 
        row)     

cursor.execute('''
    DROP VIEW IF EXISTS Districts
    ''')

#Join two tables on leadid
cursor.execute('''
    CREATE VIEW Districts AS 
        SELECT 
            y.leaid,
            year,
            NAME as name, 
            CITY as city, 
            state_name,
            LAT as lat, 
            LON as lon,
            ppcstot,
            predcost,
            fundinggap,
            outcomegap,
            enroll
        FROM
        DistrictsLocation l JOIN DistrictsYear y ON l.LEAID = y.leaid
        ''')

""" cursor.execute('''SELECT * FROM Districts''')
rows = cursor.fetchall()
with open('data/dataset_by_year.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow([i[0] for i in cursor.description])
    csv_writer.writerows(rows)

cursor.execute('''
    SELECT 
        leaid, name, city, state_name, lat, lon,
        AVG(ppcstot) AS ppcstot,
        AVG(predcost) AS predcost,
        AVG(fundinggap) AS fundinggap,
        AVG(outcomegap) AS outcomegap,
        AVG(enroll) AS enroll
    FROM
    Districts
    GROUP BY leaid, name, city, state_name, lat, lon
    ''')

rows = cursor.fetchall()
with open('data/dataset_by_avgs.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow([i[0] for i in cursor.description])
    csv_writer.writerows(rows) """

cursor.execute('''
    SELECT 
        year,
        AVG(ppcstot) AS ppcstot,
        AVG(predcost) AS predcost,
        AVG(fundinggap) AS fundinggap,
        AVG(outcomegap) AS outcomegap,
        AVG(enroll) AS enroll
    FROM
    Districts
    GROUP BY year
    ''')
rows = cursor.fetchall()
with open('data/avg_by_year.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow([i[0] for i in cursor.description])
    csv_writer.writerows(rows)


# Commit changes and close connection
conn.commit()
conn.close()


In [67]:
import plotly.graph_objects as go
import pandas as pd

df = pd.read_csv(r"data/dataset_by_avgs.csv")

name_data = df['name'].tolist()
lat_data = df['lat'].tolist()
lon_data = df['lon'].tolist()
city_data = df['city'].tolist()
state_data = df['state_name'].tolist()
fundinggap_data = df['fundinggap'].tolist()
hover_text = [f'{city}, {state}' for city, state in zip(city_data, state_data)]

thresholds = (-15000, -10000, -5000, 0, 5000)
marker_colors = []

for fundinggap in fundinggap_data:
    if fundinggap < thresholds[0]:  # Example condition
        marker_colors.append('blue')
    elif thresholds[0] <= fundinggap < thresholds[1]:  # Example condition
        marker_colors.append('blue')
    elif thresholds[1] <= fundinggap < thresholds[2]:
        marker_colors.append('red')
    elif thresholds[2] <= fundinggap < thresholds[3]:
        marker_colors.append('red')
    elif thresholds[3] <= fundinggap < thresholds[4]:
        marker_colors.append('red')
    else:
        marker_colors.append('green')

mapbox_access_token = open(".mapbox_token").read()

fig = go.Figure(go.Scattermapbox(
        lat=lat_data,
        lon=lon_data,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=9,
            color=marker_colors
        ),
        text=name_data,
        hovertext=hover_text,
        hovertemplate="<b>%{text}</b><extra>%{hovertext}</extra>"
    ))

fig.update_layout(
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=39.8283,
            lon=-98.5795
        ),
        pitch=0,
        zoom=2.5
    ),
)

fig.show()


In [66]:
import plotly.graph_objects as go
import plotly.express as px

import pandas as pd

df = pd.read_csv(r"data/dataset_by_avgs.csv")
avg_fundinggap = df['fundinggap'].mean()

name_data = df['name'].tolist()
lat_data = df['lat'].tolist()
lon_data = df['lon'].tolist()
city_data = df['city'].tolist()
state_data = df['state_name'].tolist()
fundinggap_data = df['fundinggap'].tolist()
hover_text = [f'{city}, {state}' for city, state in zip(city_data, state_data)]

mapbox_access_token = open(".mapbox_token").read()

fig = px.scatter_mapbox(df, lat="lat", lon="lon", color="fundinggap", size="ppcstot",
                  color_continuous_scale=px.colors.diverging.Portland,color_continuous_midpoint=avg_fundinggap, size_max=15, zoom=5)

fig.show()